<a href="https://colab.research.google.com/github/hvandermarel/Colab-Notebooks/blob/main/CTB3310_A4_Colab2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CTB3310 Assignment 4 - GNSS Data Analysis (Colab version)

**Hans van der Marel, Delft, 5 Mar 2024 - edited 22 March 2024 (CT/BvN) - edited 5 Mar 2025 (CFL)**

In this assignment you will analyze the GNSS data that you collected with the help of the SW Maps app during the fieldwork outdoors. The data is stored in geopackage format which we are going to read and process in Python with the help of the GeoPandas package.

## 0. Getting started

Before starting the GNSS data analysis a few prepatory actions are necessary.

## 0.1 Google Colab requirements

The following files are required for this assignment

- This Jupyter notebook with the accompanying files `nlgeo2018.gtx` and `rdcorr2018.gsb`, and three example geopackage files, which will be downloaded from Github by this notebook,
- geopackage file you created during the outdoor practical, which will be downloaded from Github by this notebook,
- answer sheet template (MS Word), available from Brightspace.

One important caveat to remember while using Colab is that the files won’t be available forever. **Colab is a temporary environment with an idle timeout of 90 minutes and an absolute timeout of 12 hours**. This means that the runtime will disconnect if it has remained idle for 90 minutes, or if it has been in use for 12 hours. On disconnection, you lose all your variables, states, installed packages, and files and will be connected to an entirely new and clean environment on reconnecting.

We therefore **strongly recommend** to **save your notebook regularly** to your Google drive
> `File > Save a copy in Drive`    (first time)

> `File > Save`  

or download it to your local drive.

Only the notebook needs to be saved to Google drive. The other files are read only and can be downloaded from Github whenever necessary.

## 0.2 Download datasets and geopackage files

All the required files and datasets collected by the students are downloaded from Github. This must be done once every time a new clean session in Google Colab is started.

In [1]:
!wget -O CTB3310_A4_GPKG2025.zip https://github.com/hvandermarel/Colab-Notebooks/blob/main/datasets/CTB3310_A4_GPKG2025.zip?raw=true
!unzip -o CTB3310_A4_GPKG2025.zip

--2025-03-11 10:52:21--  https://github.com/hvandermarel/Colab-Notebooks/blob/main/datasets/CTB3310_A4_GPKG2025.zip?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/hvandermarel/Colab-Notebooks/raw/refs/heads/main/datasets/CTB3310_A4_GPKG2025.zip [following]
--2025-03-11 10:52:21--  https://github.com/hvandermarel/Colab-Notebooks/raw/refs/heads/main/datasets/CTB3310_A4_GPKG2025.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hvandermarel/Colab-Notebooks/refs/heads/main/datasets/CTB3310_A4_GPKG2025.zip [following]
--2025-03-11 10:52:22--  https://raw.githubusercontent.com/hvandermarel/Colab-Notebooks/refs/heads/main/datasets/CTB3310_A4_GPKG2025.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.

The student geopackage files are stored in three subdirectories.

You can see all available files using the filebrowser in the navigation menu on the left (file icon), or using bash shell commands in any of the code cells, as is shown in the following example:

In [2]:
!pwd
!ls -l

/content
total 31604
-rw-r--r-- 1 root root 18890752 Mar  3  2023 Benelux.gpkg
-rw-r--r-- 1 root root 10255170 Mar 11 10:52 CTB3310_A4_GPKG2025.zip
drwxr-xr-x 2 root root     4096 Mar 11 10:52 do20feb2025
drwxr-xr-x 2 root root     4096 Mar 11 10:52 ma17feb2025
-rw-r--r-- 1 root root   277479 Feb 27 12:28 Monday.jpg
-rw-r--r-- 1 root root   579164 Mar 22  2019 nlgeo2018.gtx
-rw-r--r-- 1 root root    81920 Feb  3  2023 PracticalTest1.gpkg
-rw-r--r-- 1 root root    61440 Feb 17  2023 PracticalTest2.gpkg
-rw-r--r-- 1 root root   114688 Feb 20  2023 PracticalTest3.gpkg
-rw-r--r-- 1 root root  1477616 Mar  5  2019 rdcorr2018.gsb
drwxr-xr-x 1 root root     4096 Mar  7 14:26 sample_data
-rw-r--r-- 1 root root   286036 Feb 27 12:28 Thursday.jpg
-rw-r--r-- 1 root root   312408 Feb 27 12:28 Wednesday.jpg
drwxr-xr-x 2 root root     4096 Mar 11 10:52 wo19feb2025


## 0.3 Install GeoPandas

GeoPandas is written in pure Python, but has several dependencies written in C (GEOS, GDAL, PROJ).

Geopandas, and several dependencies, are not installed by default in Google Colab. So, this has to be done everytime a new clean session is started in Colab. Installion is done with `pip`.

In [3]:
!pip install geopandas mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.5 MB/s eta 0:00:00


## 1. Import packages and locate your data

## 1.1 Import packages

GeoPandas is an open source project to make working with geospatial data in Python easier. GeoPandas extends the datatypes used by `pandas` to allow spatial operations on geometric types. Geometric operations are performed by `shapely`. Geopandas further depends on `fiona` for file access, `pyproj` for coordinate transformations, `matplotlib` for plotting and a number of other packages.

For much of the work a simple import of `geopandas` is sufficient. The other packages do the work under the hood. However, sometimes, for more advanced work, it is necessary to import other packages in order to get access to more functionality form their native API's. Furthermore, for computational work rely on `numpy`.

In [ ]:
import geopandas
import fiona
import pyproj

import numpy as np
import matplotlib.pyplot as plt
import os

The geopandas user guide can be found here https://geopandas.org/en/stable/docs/user_guide.html. More documentation is available from https://geopandas.org/en/stable/docs.html .

## 1.2 Folder structure, accompanying and geopackage files

This Jupyter notebook requires the RD/NAP grid correction files `nlgeo2018.gtx` and `rdcorr2018.gsb`, and three example geopackage files (`PracticalTest#.gpkg`), to be installed in the **same directory** as the notebook.

To check the contents of the directories you can use your favourite file browser, the file browser on the left panel (in case of Jupyterlab) or call shell commands from within the notebook by preceeding them with an exclamation mark (!).

An example of using shell commands on Windows (on linux and Mac use `!ls`) is given below:

In [ ]:
!dir

## 1.3 Locate your geopackage data file ##

The geopackage files that were created by the students are stored in three subdirectories, called `ma17feb2025`, `wo19feb2025` and `do20feb2025`.

### Do this yourself ###

**One of the first things to do is to locate YOUR data file and assign the name of this file to a variable called `mydata`**

To see the available geopackage files for a specific day use something like this

In [ ]:
!dir wo19feb2025 # check the content of the folder wo19feb2025

# If you want to read folder ma17feb2025 or do20feb2025, you can change to !dir ma17feb2025 or !dir do20feb2025

Do you recognize the geopackage file that was created by your group during the practical?

If so, assign the name of the file, including the relative path, to the variable `mydata` like this

In [ ]:
# TODO: Replace with the correct dataset file name and uncomment the appropriate line
# mydata_day = "wo19feb2025"  # Update this with the correct dataset name and uncomment
# mydata_name = "WO17.gpkg"  # Update this with the correct file name and uncomment

# Combine 'mydata_day' and 'mydata_name' to form the full dataset path
mydata = os.path.join(mydata_day, mydata_name)

If you cannot find your dataset, or if there are problems with it, or just out of curiosity, you may also select another dataset.

**Question 1:** Write down the name(s) of the geopackage file(s) you are going to use in your report.

## 2. Read geopackage file

The GNSS data collected by SW Maps is stored in a standard GIS format: a geopackage, which we have to read.

This notebook comes with an example geopackage file called `PracticalTest2.gpkg`, to demonstrate the processing, analysis and visualization of GNSS position data. You may use this package initially as a reference and study the material, however, you MAY NOT use this for your answers. We will first show you how to read the example data. After that, you will replace the example dataset with **your own data**.

The example geopackage name is provided in the variable `exampledata`.

In [ ]:
exampledata = "PracticalTest2.gpkg"       # example dataset

A geopackage contains multiple layers. If you used the provided CTB3310 SW Maps template, the layers are: *points*, *curves* and *Tracks*. Each layer is read into a *GeoDataFrame* object (note: older versions of Python may give harmless warnings which you may ignore, if you get other warnings or errors, please consider the hints given below the next code box).

If you know the names of the layers your can read them using these commands:

In [ ]:
points = geopandas.read_file(exampledata, layer='points')
lines = geopandas.read_file(exampledata, layer='curves')
tracks = geopandas.read_file(exampledata, layer='Tracks')

The above commands assume that you stored the data in three layers named *points*, *curves* and *Tracks*; these are stored in *GeoDataFrame* objects with the names *points*, *lines* and *tracks* (Note: SW Maps uses *Tracks* as default layer name, however, we call the object *tracks*, without capitals, which is more Pythonic). If one layer is missing, or if you used different names for the layers, this will result in an error (your own dataset may not contain points).

To find out the names of the layers in the geopackage file we use `fiona`. If you find you have used different names, adjust the above commands accordingly.

In [ ]:
layers = fiona.listlayers(exampledata)
layers

To display the contents of the layers you can use print(object_name) or just type the object name (as last code line), e.g.:

In [ ]:
points

In [ ]:
lines

In [ ]:
tracks

Each layer consists of an index, feature data and **geometry**. The geometry is in Well-Known-Text (WKT) format. In our dataset two fundamental geometries are used in their three dimensional variant: **point** (obviously for points) and **linestring** (for lines/curves and tracks). The WKT representation is
```
POINT Z(<lon> <lat> <hgt>)
LINESTRING Z(<lon> <lat> <hgt>, <lon> <lat> ... )
```
The **linestring** consists of a sequence of  coordinate tuples, the **point** has exactly one. The three dimensional coordinate **tuples** `(<lon> <lat> <hgt>)` provide the geographic **longitude** &lambda; and **latitude** &phi; in *degrees* (and in this order), and **height** *h* above the ellipsoid in *meters* (also referred to as elevation).
The other columns contain the attributes that were stored with the data when it was collected, such as the name and type of the feature that you entered when collecting the data, while others, such as the fix_id (4 is RTK fixed) provided by the software.

**You should only use RTK fixed data (fix_id equal to 4). If you have used the appropriate setting (Minimum Fix Quality) in SW Maps while collecting the data this is guaranteed, if not, you may occasionally have data which is not RTK fixed (and positions may be (far) less precise).** RTK fixed data means that the carrier phase cycle ambiguities ('the integer number of full cycles/wavelengths between the satellite and the receiver') have been resolved as integer parameters, and hence the high mm-cm precision of the carrier phase measurements gets unlocked.

The `android_metadata` layer does not contain much useful information.

### Do this yourself ###

Repeat the above commands on your own dataset (For the file names see also section 0.2).

First check, using fiona, that the layer names are the ones that have been defined in the template used by SW Maps. These should be the same as in the example. Then load the layers and display the contents. The commands are already prepared, but you may have to adjust these to your situation, and uncomment the lines. We use new layer names, do not overwrite the layers from the example; we still need those.

In [ ]:
# TODO: Complete the line of code to read the layers in the dataset 'mydata'
mylayers =  # Fill in the correct function to get the layers from 'mydata'

mylayers  # print mylayers to check the layers in the dataset mydata

In [ ]:
# TODO: Uncomment and modify the line if your dataset 'mydata' contains a 'points' layer
# mypoints =

# TODO: Uncomment and modify the line for datasets 'mydata' that should include a 'curves' layer
# mylines =

# TODO: Uncomment and modify the line for datasets 'mydata' that should include a 'Tracks' layer
# mytracks =

# TODO: Uncomment and modify the line for datasets 'mydata' that should include a 'TrackPoints' layer
# mytrackpoints =

In [ ]:
mypoints

In [ ]:
mylines

In [ ]:
mytracks

In [ ]:
mytrackpoints

The layer **TrackPoints** is a recent addition by SW Maps that was not yet present in the example dataset. This layer contains the same data as the layer **Tracks**, but now stored as points with some additional (and very useful) attributes.

Another recent addition by SW Maps are the *accuracy*, *speed* and *bearing* attributes in the **points** and **trackPoints** layers. The *accuracy* attribute is a welcome addition to the *fix_id* attribute.

To find out the names in the **trackPoints** layer you can do something like this

In [ ]:
mytrackpoints.name.unique()

Unfortunately, the *fix_id* and *accuracy* are not stored as attributes in the **curves** (lines) layer; so unless you have set the *Minimum fix quality* in SW Maps to "RTK fix" you cannot be sure this data has the required quality.

**Question 2:** What is the position accuracy of your data, and is all data "RTK fixed"? Mention this in your report. If there are any surprises mention this as well.

### Check your data-file

This stage is a good moment to **check the content** of your own data-file ('mydata'), before proceeding to the rest of this assignment.

Your data-file should contain (at least) two tracks, the one for the grid (DEM) and the one with the tripod at the benchmark. And your data-file should contain some curves/lines (you should have some elements to map, such as a bicycle lane or a pedestrian path).

If this is not the case you may want to use another data-file instead.

## 3. Mapping and Plotting Topography

Mapping and plotting your data is often one of the first things to do.

## 3.1 Plotting layers and tracks

GeoPandas provides a high-level interface to the `matplotlib` library for making maps. Mapping shapes is as easy as using the `plot()` method on a GeoSeries or GeoDataFrame.

In [ ]:
tracks.plot()

This is not a very good plot as labels and a title is missing, we show below how to add these.

## 3.2 Plotting multiple layers

For making a map with multiple layers, first remember to always ensure they share a common Coordinate Reference System (CRS), so they will align. In our dataset this is (still) the case as the layers come from the same source (GPS/GNSS RTK receiver). To create a map with multiple layers we call the plot method on individual layers with options.

Note that in general, any option one can pass to pyplot in matplotlib (or style options that work for lines) can be passed to the plot() method. We define an object called 'base'.

In [ ]:
base = lines.plot(color='blue')
points.plot(ax=base, marker='o', color='red', markersize=10)
tracks.plot(ax=base, color='green')
base.set(xlabel='Longitude [deg]', ylabel='Latitude [deg]', title='Example Map');

### Do this yourself ###

Recreate the above plot for your own dataset

In [ ]:
mybase = mylines.plot(color='blue')
# TODO: If your dataset `mydata` contains points and/or tracks, plot them on the map (Remember to use ax=mybase)
# mypoints.plot(...)
# mytracks.plot(...)

# Set the labels and title
mybase.set(xlabel='Longitude [deg]', ylabel='Latitude [deg]', title='Your Map');

**Question 3:** Include this plot/map in your report.

### Aspect ratio ###

The *GeoPandas* plot() method is **smart**: it takes into account the fact that - except at the equator - one degree in latitude is not the same distance in units of length (e.g. meters) as one degree in longitude!

This is not done when working with pyplot directly.

To illustrate this point we redo the previous map (the Example Map), using a slightly different method to plot multiple layers using `pyplot`, and set the **aspect ratio** to **one** (hence equal scaling of vertical and horizontal axis in terms of degrees, which is not correct).

In [ ]:
fig, ax = plt.subplots()

lines.plot(ax=ax, color='blue')
points.plot(ax=ax, marker='o', color='red', markersize=10)
tracks.plot(ax=ax, color='green')
ax.set(xlabel='Longitude [deg]', ylabel='Latitude [deg]', title='Incorrect Map')

ax.set_aspect(1)

plt.show();

Clearly, this is not the way longitude and latitude data should be plotted (you have been warned now!). When you plot longitude and latitude data always set the appropriate aspect ratio.

With plotting/mapping, on a flat piece of paper or a screen, we turn a difference in longitude $d\lambda$ into a distance $dE$, and a difference in latitude $d\varphi$ into a distance $dN$, see section 29.2.2 of the book on Surveying and Mapping. Equation (29.7) shows how to do this, with $d\varphi$ and $d\lambda$ expressed in radians. Look carefully, and in the second expression, for $dE$, you see an extra term, namely $cos(\varphi)$. This term accounts for the fact that at higher latitudes, one degree of longitude corresponds to a smaller distance than at the equator ($\varphi=0$); the 52-degrees North latitude circle is a much smaller circle than the equator. In mapping we take this into account through the aspect ratio.

### Do this yourself ###

The aspect ratio can be computed using formula (29.7) from the Surveying and Mapping book, section 29.2.2, whereby you may assume that the curvatures are equal ($\bar{N}(\varphi) = \bar{M}(\varphi)$), and $h=0$ (hence, at the surface). You can also determine the aspect ratio from the plot in Question 3 by measuring, with a ruler, the length in the map corresponding to 0.0001 deg in latitude and the length in the map to 0.0001 deg in longitude.  

In [ ]:
# TODO: Compute the ratio from Equation (29.7), assuming M(phi) and N(phi) are equal
ratio_computed =                          # Fill in the correct formula

# TODO: Calculate the measured ratio in the map (replace numerator and denominator with actual values)
ratio_measured_in_map =   /               # Replace with the correct values (mm per 0.0001 deg latitude/longitude)

# Print the computed and measured ratios for comparison
print('ratio computed:', ratio_computed, ', measured in map:', ratio_measured_in_map )

Hint: if you don't have a ruler, use a snipping tool (e.g. snip&sketch in Windows) with a built in ruler.

**Question 4:** Explain in your report why the aspect ratio is not equal to one, and report the aspect ratio computed using formula (29.7) in the Surveying and Mapping book and the aspect ratio measured using a ruler from the plot in Question 3.

Check the computed (and measured) aspect ratio by adjusting the aspect ratio in the previous plot of the Example Map, using the ratio computed in Question 4.  You do not have to give the plot in your report (but it is a good check to your answer to Question 4):

In [ ]:
fig, ax = plt.subplots()

lines.plot(ax=ax, color='blue')
points.plot(ax=ax, marker='o', color='red', markersize=10)
tracks.plot(ax=ax, color='green')
ax.set(xlabel='Longitude [deg]', ylabel='Latitude [deg]', title='Example Map')

# TODO: Replace the aspect ratio with the correct value for your dataset
#ax.set_aspect(...)

plt.show();

However, if you want to do geometric computations, such as computing areas, it is a much better idea to use a map projection, as we will explore later (in section 7).

## 3.3 Interactive mapping

GeoPandas can also create interactive maps based on the `folium` library (under the hood in GeoPandas). Creating maps for interactive exploration uses the `explore()` method of a GeoSeries or GeoDataFrame.

The `explore()` method returns a `folium.Map` object, which can also be passed directly (as you did with ax in plot()). In the example below, we plot our three layers on the same map by passing the folium.Map object to subsequent layers (using *m* instead of *ax* keywords).

It may take some time to generate the interactive map, so be patient, and also you may be asked to "make this notebook trusted" to load the map (answer yes). We define an object called 'fm'.

In [ ]:
fm = lines.explore(color='blue')
points.explore(m=fm, color='red')
tracks.explore(m=fm, color='green')

Hovering over a feature shows a pop-up with all the feature attributes.

### Do this yourself ###

Create the interactive map for your dataset ('mydata').

In [ ]:
# TODO: Create an interactive map using the explore function
# Start with the explore function in 'mylines' layer
fm = mylines.

# TODO: Uncomment and use the explore function to add 'mypoints' to the map if your dataset contains points
# mypoints.

# TODO: Uncomment and use the explore function to add 'mytracks' to the map
#mytracks.

 You don't have to include this in your report.

**These interactive maps, and the pop-ups created by hovering over a feature, are very useful to identify features in your dataset that you may need for further analysis. So, when you need to identify a feature in your dataset for one of the questions, you can come back to this interactive map to find it!**

After importing folium you can also use folium functionality directly on the resulting map, e.g. adding additional tiles allowing you to change the background directly in the map. We leave this to you to experiment (after consulting GeoPandas and folium documentation).

Note that to display the background tiles in the correct position, and to get the aspect ratio correct as with the static matplotlib plots, the correct Coordinate Reference System (CRS) must have been set in the GeoDataFrame. This is something we deal with in the next section.

> ### A note on interactive mapping (QGIS promotion, for your information)
>
> Interactive mapping may seem a nice and useful feature of GeoPandas. However for truely interactive mapping we recommend using a GIS system, such as ArcGIS or the freely available **QGIS** that will be used in assignment 5 and 6.
> To obtain more or less the same result in QGIS:
> 1. open a new QGIS project,
> 2. double click `browser > XYZ Tiles > OpenStreetMap`,
> 3. add your geopackage file using `Layer > Add Layer > Add Vector Layer` .
>
> Do not forget to set your project CRS to e.g. EPSG:28992 (otherwise you have this very unprofessional distorted longitude/latitude reference system.   

## 4. Coordinate Reference System (CRS)

A GeoDataFrame needs to have a Coordinate Reference System (CRS) set to perform spatial analyses and enable coordinate transformations.

## 4.1 Obtaining CRS information

To obtain information on the CRS use the `.crs` attribute on a GeoDataFrame or GeoSeries.

In [ ]:
lines.crs

In [ ]:
print(lines.crs)

An important element is the EPSG code. Each geodetic datum, spatial reference system, Earth ellipsoid, coordinate transformation and related units of measurements is assigned an EPSG code between 1024 and 32767, that corresponds to a public registry in the EPSG Geodetic Parameter Dataset, originated by a member of the European Petroleum Survey Group (EPSG) in 1985, and maintained by the IOGP Geomatics Committee, see also Section 31.4 in the book.

Common EPSG codes are

* EPSG:4326 - WGS 84, latitude/longitude coordinate system based on the Earth's center of mass, used by the Global Positioning System (GPS) among others.
* EPSG:3857 - Web Mercator projection used for display by many web-based mapping tools, including Google Maps and OpenStreetMap.
* EPSG:7789 - International Terrestrial Reference Frame 2014 (ITRF2014), an Earth-fixed system that allows for tectonic plate motion; continents may move over time.
* EPSG:4258 - European Terrestrial Reference Frame 1989 (ETRS89), an Earth-Fixed system moving along with the Eurasian Plate, defined only for the European continent.
* EPSG:28992 - Projected coordinate system for Netherlands - onshore, including Waddenzee, Dutch Wadden Islands and 12-mile offshore coastal zone.

These systems are covered later in class and in part V of the book.

Our data is apparently in WGS84, but as we will find out later, this is not entirely correct (to be precise: it is the national realization of ETRS89). Coordinates are given as longitude &lambda;, latitude &phi; and elevation (height) *h* above the ellipsoid, cf. Figure 29.2 of the book (mind that the order of the coordinates is specifically as given here!).

Another way to describe the CRS that is used is by the standard machine-readable well-known text (WKT) representation using the *to_wrt()* method:

In [ ]:
print(lines.crs.to_wkt(pretty=True))

## 5. Data selections with GeoPandas (and Pandas) and conversion to numpy

This section is a short tutorial on (Geo)Pandas using only the example dataset.

So far we were not very specific in what we plotted. However, for further analysis of the data, we have to be able to select specific features (e.g. a point or track) that we want to analyze. How to do these selections, and how to convert the selected data into a `numpy` array, is explained below.


## 5.1. Selecting rows (features and tracks)

GeoDataFrames are an extension of Panda's dataframes. This means you can select rows using the `loc[]`, `iloc[]` and `query()` methods. This is best ilustrated with examples:

In [ ]:
lines.iloc[[0]]

In [ ]:
lines.iloc[[0,1]]

In [ ]:
lines.loc[[0,1]]

In [ ]:
lines.query( "type == 'road'" )

The `loc[]` and `iloc[]` methods produce similar results, this is because the *index* is numerical. We can set any column as index, e.g.

In [ ]:
points_idx = points.set_index('name')
points_idx

In [ ]:
points_idx.loc[['a1']]

The type that is returned is always a GeoPandas geodataframe (Note: if single `[]` are used, with one row, a Pandas series is returned)

In [ ]:
print(type(lines.loc[[0]]))

## 5.2 Selecting columns and geometries

Specific columns can be selected as well, using e.g. the following syntax

In [ ]:
points_idx[['longitude', 'latitude']]

There is purpose behind using `[[ ]]` or `[]`. The first returns a Pandas dataframe, the second a Pandas series:

In [ ]:
print(type(points_idx[['latitude']]))
print(type(points_idx['latitude']))

The geometry is something special. If this is included a GeoPandas GeoDataFrame or GeoSeries is returned:

In [ ]:
print(type(points_idx[['geometry']]))
points_idx[['geometry']]

In [ ]:
print(type(points_idx['geometry']))
points_idx['geometry']

## 5.3 Converting geometries into numpy arrays with coordinates  

In the previous section the geometry was retrieved in the typical Pandas way using a column selection. This is so important that this has been implemented in GeoPandas as `geometry` method.
This method is used, together with other methods, to retrieve the coordinates as `numpy` arrays for further analysis.

We show this with a couple of examples.

The `geometry` method returns what is called a *GeoSeries* (the equivalent of a Pandas data series).

In [ ]:
print(type(points_idx.geometry))
points_idx.geometry

To obtain the `wkt` representation as string the *wkt* method can be used on a single instance of a *geometry* object:

In [ ]:
print(type(points_idx.geometry[0].wkt))
points_idx.geometry[0].wkt

To obtain the coordinates use `coords` (from the `shapely` package) on a single geometry instance, and then convert to a list with coordinate *tuples* or a two-dimensional numpy *array*:

In [ ]:
print(type(points_idx.geometry[0].coords))
print(list(points_idx.geometry[0].coords))
np.array(points_idx.geometry[0].coords)

Doing this on LINESTRING returns a fully populated two-dimensional numpy array:

In [ ]:
np.array(lines.geometry[0].coords)

Having the coordinates as numpy array means that you can do a lot of other stuff on the data and perform analyses. For example, you can compute the mean and standard deviation of the repeated measurements on the static point (benchmark), in order to evaluate or validate the data quality.

## 6. Analysis of the benchmark measurements

## 6.1 Why benchmarks?

It is always good practice when you go out into the field to measure in addition to your data at least two existing points (benchmarks) with **known** coordinates. The reason is that you can thus check your measurements for big mistakes, consistency with previous measurements and to add realibility to the measurement campaign. When you come back another day, you certainly want to be sure that the new measurements match the previous ones, and that they are compatible with other projects.

Also, whenever you start a new project, it is strongly recommended to create a few new benchmarks (points) that you can revisit on subsequent visits to the site (not having to travel kilometers to a "far far away" national benchmark). To create a new benchmark it is custom to measure for a longer time and to average over a period of time, so that you get a slightly better precision for the new coordinates (this is an option in SW Maps).

This was done in example dataset.

However, we did ask you - for educational purposes - to do this a bit differently:

* Record data for several minutes with the range pole stable on one of the (known) benchmarks (P1, P2, P3, P4) using a small tripod, this data is recorded as a **track**

The purpose of this experiment is to get an idea of the quality of the position coordinates resulting from your GNSS survey, specifically we consider the spread/variability (precision) in the positions due to GNSS measurement noise, as well as any bias, and the overall accuracy.

In the example dataset this was the first track (but this  may be different in yours!); we set this in the variable `benchmark_trackidx`.

In [ ]:
tracks

In [ ]:
benchmark_trackidx=0

Below we first check the content of the benchmark track (how many entries with 3 coordinates there are), ready for analysis.

In [ ]:
static_point_coords = np.array(tracks.geometry[benchmark_trackidx].coords)
print('shape',static_point_coords.shape)

## 6.2 Known benchmark coordinates

The coordinates of the benchmarks **P1**, **P2**, **P3** and **P4** are very well known from dedicated measurement campaigns.  These coordinates are used to compare with your own data and to make sure that your data is consistent with other measurements.

The coordinates for **P1**, **P2**, **P3** and **P4** can be obtained from the **TU Delft GNSS Fieldlab** webpage
http://gnss1.tudelft.nl/dlab/station/GNSS_Fieldlab_Coordinates.html .

Coordinates are given as **latitude, longitude (mind, this order!!)** and height are in ETRS89 (Dutch national realization based on ETRF2000), and in RD and NAP (Normaal Amsterdams Peil) as well (x,y,H).

The RD and NAP coordinates have been computed using the official RDNAPTRANS(TM) procedure using PCTrans from the ministry of Defense (https://www.defensie.nl/downloads/applicaties/2021/06/30/pctrans5_20210630). This coordinate transformation is centimeter accurate, as opposed to the one provided by `EPSG:28992`. In section 7.2 we shall also do a more accurate transformation using `PROJ` in Python.

### Do this yourself ###

Go to the GNSS Fieldlab webpage and save the **longitude, latitude and ellipsoidal height (in this order!!)** in the array `ref_llh` and the RD/NAP coordinates in the array `ref_rd`

In [ ]:
# TODO: Fill in the appropriate reference name ('P1', 'P2', 'P3', or 'P4') inside the quotes
ref_name = ['']

# TODO: Define the reference coordinates in LLH format
# Order: longitude (deg), latitude (deg), ellipsoidal height (m)
ref_llh = np.array([, , ])

# TODO: Define the reference coordinates in RD format
# Order: x-RD, y-RD, H-NAP
ref_rd = np.array([, , ])

We are going to compare these to the values of the benchmark-tripod measurements.

## 6.3 Conversion to units of length

Next, we prepare for the analysis by introducing the conversion of small differences in longitude and latitude into units of length, specifically dE and dN, see section 29.2.2 in the book (as small differences in longitude and latitude in degrees or radians are hard to interpret). Measures expressed in meter are easier to interpret.

First, we get the semi-major axis (*a*) and inverse flattening (*finv*) for the ellipsoid from one of the GeoDataFrames, and compute some other ellipsoid parameters that are needed later on:

In [ ]:
crscf = points.crs.to_cf()
a = crscf['semi_major_axis']
finv = crscf['inverse_flattening']
f = 1/finv
e2 = 2*f - f**2

print('a, finv:', a, finv)

Then we compute the scaling factors (deg -> meters) for the latitude and longitude using eqs. (29.6) for the radius of curvature and the conversion factors from (29.7). We use the exact expressions for the radii here, though for most practical applications just setting $\bar{N}=\bar{M}=R=6378000$ m will be fine.

As the reference latitude we obviously use the given/known latitude of the benchmark.

In [ ]:
lat0=ref_llh[1]                  # reference latitude in degrees (52 degrees)
lat0rad = np.deg2rad(lat0)       # reference latitude in radians

N = a / np.sqrt(1 - e2 * np.sin(lat0rad)**2)   # radius of curvatures, h=0 taken here (hence, on the Earth's surface)
M = N * (1 -e2) / ( 1 - e2 * np.sin(lat0rad)**2)
flat= M * np.pi / 180                          # conversion factors dphi [deg] -> dN [m]
flon= N*np.cos(lat0rad) * np.pi / 180          #                    dlambda [deg] -> dE [m]

print('lat, flat, flon:', lat0, flat, flon)

In this conversion the height above the ellipsoid is ignored (h=0), therefore, the resulting metrics are on the surface of the ellipsoid.

## 6.4 Benchmark analysis

We compute the mean of the points in the benchmark track, also the standard deviation and the Root Mean Square (rms) error, per coordinate.

In case you do not happen to have a known benchmark, you can still compute the standard deviation and evaluate the precision (but obviously, then you cannot evaluate the bias and the rms).

In [ ]:
tripod_mean_llh = static_point_coords.mean(axis=0)
tripod_stdev_llh = static_point_coords.std(axis=0, ddof=1)

print('mean',tripod_mean_llh, 'in deg/deg/m')
print('stdev',tripod_stdev_llh, 'in deg/deg/m')

The mean is compared with the known (ref) coordinate, as to evaluate whether there is a bias in the position solution (see section 6.6 in the book). Below, first the bias in the three coordinates is presented in longitude, latitude, height in deg/deg/m, which is hard to interpret, and next all in units of length, in m/m/m.

In [ ]:
diff_llh = tripod_mean_llh - ref_llh
print('Difference in Longitude and Latitude (mean-ref): ', diff_llh[:2], ' deg')
print('Difference in Height (mean-ref): ', diff_llh[2], ' m')

In [ ]:
diff_enu = diff_llh.copy()
diff_enu[0] =  diff_llh[0]*flon
diff_enu[1] =  diff_llh[1]*flat

print('Difference (mean-ref) in East, North and Up in m:\n', diff_enu)

Next, we will compute the difference per coordinate for all points in the track, with respect to the given known coordinate values, and express these differences in dE, dN and dh. So, we create a time series of dE, dN and dh coordinates of the entire track and plot them.

In [ ]:
static_point_coords_enu = (static_point_coords - ref_llh) * np.array([flon, flat, 1])

fig, ax = plt.subplots(1,2, figsize=(16,8))

ax[0].scatter(static_point_coords_enu[:,0], static_point_coords_enu[:,1], label='Measurements')
ax[0].set_xlabel('East [m]')
ax[0].set_ylabel('North [m]')
ax[0].set_title('Scatter plot')
#ax[0].set_xlim([-0.05, 0.05])
#ax[0].set_ylim([-0.05, 0.05])
ax[0].grid()

ax[1].plot(np.arange(len(static_point_coords_enu[:,2])), static_point_coords_enu[:,2], label='Measurements')

ax[1].set_xlabel('Time Epoch [-]')
ax[1].set_ylabel('Height [m]')
ax[1].set_title('Time series of height coordinate')
#ax[1].set_ylim([-0.05, 0.05])
ax[1].grid()


Finally we compute the standard deviation, as a measure of precision, per coordinate (with respect to their mean), and we compute the root mean square error, as the measure of accuracy, per coordinate (using the coordinate differences dE, dN, dh which were computed with respect to the given known coordinates of the benchmark), using Eq (6.18).

In [ ]:
tripod_stdev_enu = static_point_coords_enu.std(axis=0, ddof=1)
print('stdev',tripod_stdev_enu, 'in m')

In [ ]:
MSE_enu = (static_point_coords_enu**2).mean(axis=0)
rms_enu = np.sqrt(MSE_enu)
print('rms in East, North and Up in m:\n', rms_enu)

### Do this yourself ###

Repeat the above computations for your own dataset (`mydata`, `mytracks`).

* first identify the index number of the appropriate track and set the value in `my_benchmark_trackidx`
* compute the means (bias as well) and standard deviations
* create the dEast-dNorth scatter plot and the time series of the height dh
* and compute the rms

The reference coordinates `ref_llh` and `ref_rd` have been set before already in section 6.2 (with the example dataset, make sure they match your dataset), and also the `flat` and `flon` factors have been computed already.

In [ ]:
mytracks

In [ ]:
# TODO: Replace the 'my_benchmark_trackidx' with the correct track index for your dataset
my_benchmark_trackidx=0

In [ ]:
# TODO: Extract the coordinates of the benchmark track index from 'mytracks'
mystatic_point_coords =

# Print the shape of the extracted coordinate array
print('shape', mystatic_point_coords.shape)

# TODO: Compute the mean position (longitude, latitude, height)
mytripod_mean_llh =

# TODO: Compute the standard deviation of the position
mytripod_stdev_llh =

# Print the computed mean and standard deviation values in deg/deg/m
print('mean', mytripod_mean_llh, 'in deg/deg/m')
print('stdev', mytripod_stdev_llh, 'in deg/deg/m')


In [ ]:
# TODO: Compute the difference between the mean coordinates and reference coordinates (LLH)
mydiff_llh =

# Print the difference in Longitude and Latitude (mean - reference)
print('Difference in Longitude and Latitude (mean-ref): ', mydiff_llh[:2], ' deg')
# Print the difference in Height (mean - reference)
print('Difference in Height (mean-ref): ', mydiff_llh[2], ' m')

In [ ]:
# TODO: Create a copy of the difference in LLH coordinates to work with ENU coordinates
mydiff_enu =
# TODO: Convert the difference in longitude to East (multiply by the scaling factor 'flon')
mydiff_enu[0] =
# TODO: Convert the difference in latitude to North (multiply by the scaling factor 'flat')
mydiff_enu[1] =

# Print the difference in East, North, and Up in meters (converted from LLH)
print('Difference (mean-ref) in East, North and Up in m:\n', mydiff_enu)

In [ ]:
# TODO: Convert the static point coordinates from LLH to ENU using the reference LLH and scaling factors
mystatic_point_coords_enu =

# TODO: Create a figure and axis for two subplots (scatter plot and time series)
fig, ax = plt.subplots(1,2, figsize=(16,8))

# TODO: Create the scatter plot for East and North differences (ENU coordinates)
# ax[0].scatter(...)

# TODO: Create the time series plot for the Height coordinate (Up) differences
# ax[1].plot(...)

In [ ]:
# TODO: Compute the standard deviation of the ENU coordinates (East, North, and Height)
mytripod_stdev_enu =

# Print the computed standard deviation in meters
print('stdev', mytripod_stdev_enu, 'in m')

In [ ]:
# TODO: Compute the Mean Squared Error (MSE) for the ENU coordinates (East, North, and Height)
myMSE_enu =

# TODO: Compute the Root Mean Square (RMS) for the ENU coordinates
myrms_enu =

# Print the computed RMS values in meters for East, North, and Up
print('rms in East, North and Up in m:\n', myrms_enu)

**Question 5:** Create a table, like Table 15.1 in the book, with mean (bias), std and rms, with the appropriate number of significant digits (think about this), and include it in your report. Are these standard deviations what you would expect from `RTK-fixed` solutions?

**Question 6:** Include the scatterplot in your report (make sure it is for your dataset!) and compare this to the standard deviations of Question 5.  What are your conclusions, are the coordinates correlated? Also include the time series plot of the height (comparable to Figure 15.5 in the book).

## 7 Coordinate Reference System (CRS) transformation

In this section we cover the Coordinate Reference System (CRS) transformation and map projection for the Netherlands. We first cover an approximate, meter-accurate transformation (with GeoPandas), and next the official cm-accurate RDNAPTRANS procedure.

## 7.1 Coordinate Reference System (CRS) transformation with GeoPandas

Coordinate Reference System (CRS) transformations with GeoPandas is made ''easy'' using the `pyproj` package, the Python interface to **PROJ** (cartographic projections and coordinate transformations library), under the hood.

> PROJ is a generic coordinate transformation software written in C that transforms geospatial coordinates from one coordinate reference system (CRS) to another. This includes cartographic projections as well as geodetic transformations, cf. Figure 31.1 of the book. PROJ includes an API as well as command line applications. PROJ started purely as a cartography application letting users convert geodetic coordinates into projected coordinates using a number of different cartographic projections. Over the years, as the need has become apparent, support for datum shifts has slowly worked its way into PROJ as well. Today PROJ supports more than a hundred different map projections and can transform coordinates between datums using all but the most obscure geodetic techniques. PROJ has become the de-facto standard for coordinate transformations all around the World. See also Section 31.5 of the book.

GeoPandas provides a high-level interface to PROJ using the `to_crs()` method on a GeoSeries or GeoDataFrame.

The next examples show how to transform the example GeoDataFrames to the Dutch projected coordinate system, called RD (RijksDriehoeksmeting - EPSG-code 28992), cf. Figure 35.2:

In [ ]:
points_rd = points.to_crs("EPSG:28992")
lines_rd = lines.to_crs("EPSG:28992")
tracks_rd = tracks.to_crs("EPSG:28992")

In [ ]:
lines_rd.crs

In [ ]:
points_rd

In [ ]:
lines_rd

As you can observe from the GeoDataFrames the *geometry* has changed (horizontal RD coordinate values, in Delft values around 85 km and 445 km), but not the other attributes! (the point GeoDataFrame still has attributes latitude and longitude from the original).
Also note the transformation **DID NOT change the elevation** (the third coordinate). The height coordinate is still given as height above the original ellipsoid!

Below the data is plotted in the new CRS:

In [ ]:
base = lines_rd.plot(color='blue')
points_rd.plot(ax=base, marker='o', color='red', markersize=10)
tracks_rd.plot(ax=base, color='green')
base.set(xlabel='x-RD [m]', ylabel='y-RD [m]', title='Example Map in RD');

The aspect ratio is now one (as we would expect from projected coordinates). The Dutch RD map projection is specifically meant for mapping the Netherlands.

### Do this yourself ###

Recreate the above plot for your own dataset.

In [ ]:
# TODO: Uncomment and convert 'mypoints' to the RD coordinate system (EPSG:28992) if your dataset contains points
# mypoints_rd =

# TODO: Convert 'mylines' and 'mytracks' to the RD coordinate system (EPSG:28992)
mylines_rd =
mytracks_rd =

# TODO: Create a base plot with 'mylines_rd'
mybase =

# TODO: Uncomment and plot 'mypoints_rd' on the same base plot (use ax=mybase) if your dataset contains points
# mypoints_rd.plot(...)

# TODO: Plot 'mytracks_rd' on the same base plot (use ax=mybase)
mytracks_rd.plot(...)

# Set the labels and title
mybase.set(xlabel='x-RD [m]', ylabel='y-RD [m]', title='Your Map in RD');

**Question 7: Include this map in RD in your report.**

### EPSG:28992 Limitations, BEWARE ! ###

The above coordinate transformation is very easy, but has two main limitations: it is **approximate**, and it **only** operates on the **horizontal coordinates**. This is good enough for plotting (to get an overall impression), but not enough for mathematical analysis!

> The coordinate transformation from EPSG:4326 (WGS-84) to EPSG:28992 has only an accuracy of about 1 meter. See also https://epsg.io/28992-to-4326 . This is mainly for two reasons: we actually have geographic coordinates in ETRS89 instead of WGS-84, and because a correction grid for RD was not used. In the next subsection we will show how to do a more accurate transformation, and why this is important.

We will also discuss later how to convert the height above the ellipsoid into elevation above the Amsterdam Ordance datum, in Dutch, Normaal Amsterdam Peil (NAP). This is important because there is a difference of about 42 meters between the elevation above the ellipsoid and NAP heights, cf. Figure 35.10 (check-out the height for instance of points in your track, it is typically about 42-43 m, hey, you didn't believe we were more than 40 meters above sea-level in Delft, really?).

## 7.2 RDNAPTRANS™2018 - Official Coordinate transformation between RD, NAP and ETRS89

RDNAPTRANS (https://www.nsgi.nl/rdnaptrans) is the name of the official Dutch transformation and mapping procedure between RD (Rijksdriehoeksmeting), NAP (Normaal Amsterdams Peil) on one side and ETRS89 (European Terrestrial Reference System 1989) on the other. The current version is RDNAPTRANS™2018. RDNAPTRANS is maintained by the Nederlandse Samenwerking Geodetische Infrastructuur (NSGI). This version is accurate at the cm level.

The following implementations and API are available (https://www.nsgi.nl/web/nsgi/geodetische-infrastructuur/coordinatentransformatie):

- PCTrans: A freely available Windows 10 application for geodetic and hydrographic computations from the Hydrographic Service (https://www.defensie.nl/downloads/applicaties/2021/06/30/pctrans5_20210630)
- Web based API provided by NSGI (https://www.nsgi.nl/coordinatentransformatie-api), API-key required
- Implementations of RDNAPTRANS™2018 by software providers (validated by NSGI)

RDNAPTRANS™2018 can be implemented using the Transform method from the Python-3 `pyproj` package with PROJ version 7.x or higher. Two additional files are required: the geoid file `nlgeo2008.gtx` and transformation grid file `rdcorr2018.gsb` must be present in the same folder as this module.

Not only is RDNAPTRANS more accurate than the `.to_crs('EPSG:28992')` method in the previous subsection, **but RDNAPTRANS is also able to convert ellipsoidal height into NAP heights**, something we need for our Digital Elevation Model (DEM).

Reference:

Jochem Lesparre, Lennard Huisman and Bas Alberts (2020), RDNAPTRANS™2018 - Coordinate transformation to and from Stelsel van de Rijksdriehoeksmeting and Normaal Amsterdams
Peil, Nederlandse Samenwerking Geodetische Infrastructuur, 5 Feb 2020.

#### 7.2.1 Pyproj implementation of RDNAPTRANS

RDNAPTRANS™2018 requires the Python-3 `pyproj` package with PROJ version 7.x or higher. Before continuing, check if you are using PROJ version 7.x or higher:

In [ ]:
pyproj.show_versions()

RDNAPTRANS™2018 is implemented using the `pyproj` Transform method. A `pyproj.Transformer` object `rdnaptrans2018` is created with the transformation parameters given by the variable  `pipeline_3d`. The transformer object is used to create the functions `etrs89_to_rdnap(lon, lat, elevation) -> x, y, H` and the inverse transformation `rdnap_to_etrs89(x, y, H) -> lon, lat, elevation`:

In [ ]:
# Define variables with the PROJ transform pipeline and pyproj.Transformer object

pipeline_3d = (
    '+proj=pipeline '
    '+step +proj=unitconvert +xy_in=deg +z_in=m +xy_out=rad +z_out=m '
    '+step +proj=axisswap +order=2,1,3,4 '
    '+step +proj=vgridshift +grids=./nlgeo2018.gtx '
    '+step +proj=push +v_3 '
    '+step +proj=set +v_3=43 +omit_inv '
    '+step +proj=cart +ellps=GRS80 '
    '+step +proj=helmert +x=-565.7346 +y=-50.4058 +z=-465.2895 +rx=-0.395023 '
    '+ry=0.330776 +rz=-1.876073 +s=-4.07242 +convention=coordinate_frame +exact '
    '+step +proj=cart +inv +ellps=bessel '
    '+step +proj=hgridshift +inv +grids=./rdcorr2018.gsb,null '
    '+step +proj=sterea +lat_0=52.156160556 +lon_0=5.387638889 +k=0.9999079 '
    '+x_0=155000 +y_0=463000 +ellps=bessel '
    '+step +proj=set +v_3=0 +omit_fwd '
    '+step +proj=pop +v_3'
)

rdnaptrans2018 = pyproj.Transformer.from_pipeline(pipeline_3d)

# Define function with forward transformation

def etrs89_to_rdnap(lon, lat, elevation):
    # x, y, H = etrs89_to_rdnap(lon, lat, elevation)
    return rdnaptrans2018.transform(lat, lon, elevation, direction='FORWARD')

# Define function with inverse transformation

def rdnap_to_etrs89(x, y, H):
    # lon, lat, elevation = rdnap_to_etrs89_3d(x, y, H):
    lat, lon, elevation = rdnaptrans2018.transform(x, y, H, direction='INVERSE')
    return lon, lat, elevation


Note that the `pipeline_3d` variable contains references to the geoid file `nlgeo2008.gtx` (line 4) and transformation grid file `rdcorr2018.gsb` (line 11), which are expected to be present in the same folder as this notebook. If these files are stored in a different location, you have to adjust the path in the `pipeline_3d` variable. If these files are not present in the expected location the `Transformer` object returns an error!

We test the transformation using the given reference coordinates (which you set in section 6.2); the known coordinates of the benchmark were given both in ETRS89 (lon,lat,h) and in RD-NAP (x,y,H).

In [ ]:
tmp = etrs89_to_rdnap(ref_llh[0], ref_llh[1], ref_llh[2])
print('difference (in [m]):\n', (tmp - ref_rd))

You should get differences that are smaller than one millimeter. Check!

Now lets check the inverse transformation:

In [ ]:
tmp = rdnap_to_etrs89(ref_rd[0], ref_rd[1], ref_rd[2])
print('difference (in [deg/deg/m]):\n', (tmp - ref_llh))

The two functions also operate on arrays in case you desire to transform many coordinates at once. For example, this allows us to convert height/elevation above the ellipsoid to NAP height for the Digital Elevation Model (DEM) in the next section.

## 8. Computing a Digital Elevation Model (DEM)

During the practical you collected position and elevation measurements on a grid from which you will compute a Digital Elevation Model (DEM).

The data was stored in your dataset as a track. In our example dataset this was the second track, but this may be different in yours! In that case please adapt!

In [ ]:
demidx=1

Extract this track as a numpy array (this was covered in section 5) and convert into RDNAP:

In [ ]:
dem_data_geo = np.array(tracks.geometry[demidx].coords)
print('shape',dem_data_geo.shape)

In [ ]:
xRD, yRD, HNAP = etrs89_to_rdnap(dem_data_geo[:,0], dem_data_geo[:,1], dem_data_geo[:,2])

The difference between ellipsoidal height h and NAP height H is the geoid height N: N = h - H (in [m]), see Figure 33.1. Also check that the difference value that you get (the mean over the small area that you surveyed), the geoid height N, matches the geoid-height in the Delft-area, by inspecting Figure 35.10 (left), presenting the NLGEO2018 geoid.

In [ ]:
hdif = dem_data_geo[:,2] - HNAP
print(np.mean(hdif),np.std(hdif))

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

plt.scatter(xRD, yRD ,c=HNAP, s=5)
plt.xlabel('x-RD [m]')
plt.ylabel('y-RD [m]')
plt.title('Scatter plot with NAP-height')
plt.colorbar(label='NAP height [m]')

ax.set_aspect('equal')

plt.show();

In [ ]:
from matplotlib.tri import Triangulation, LinearTriInterpolator

In [ ]:
npts=100

In [ ]:
triObj = Triangulation(xRD,yRD)      # measured locations form an irregular grid they serve as vertices (nodes) of triangular interpolation
                                        # see chapter 11 in the book on Surveying and Mapping
fz = LinearTriInterpolator(triObj,HNAP) # linear interpolation (of terrain height) within triangles

min_x = np.amin(xRD)                 # determine the boundaries of the (horizontal) area of interest
max_x = np.amax(xRD)
min_y = np.amin(yRD)
max_y = np.amax(yRD)

xnew = np.linspace(min_x,max_x,npts) # set-up new, regular grid
ynew = np.linspace(min_y,max_y,npts)
xxnew, yynew = np.meshgrid(xnew, ynew) # meshgrid to turn new 1D arrays into grid-array coordinates

znew = fz(xxnew,yynew)               # do actual interpolation to new, regular grid

fig = plt.figure()
cs = plt.contourf(xnew, ynew, znew)  # produce contour-map
cb = plt.colorbar(cs)                # add color-bar for height
plt.title('Contour-map with NAP-height [m]')
plt.xlabel('x_RD [m]')
plt.ylabel('y_RD [m]')
plt.axis('equal')

plt.show()

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
colormap = 'viridis'
surf = ax.plot_trisurf(xRD, yRD, HNAP, cmap=colormap, antialiased = True)
ax.view_init(30,330)             # set viewing angle - default (elevation=30deg, azimuth=300deg), i.e. looking from South-East
plt.title('3D terrain model with NAP height [m]')
plt.xlabel('x_RD [m]')
plt.ylabel('y_RD [m]')

plt.show();

### Earth work

Finally, now we have a nice 3D DEM, we compute the volume (in cubic meter) needed to fill the small hill up to a flat body at 0 m NAP.

The function below requires the regular, square grid created for the interpolation (xxnew, yynew) and the interpolated height values (znew) at these grid points. It will carry out a simple numerical integration, and use for each square cell the mean height value of the four grid points. It will do so for just the surveyed area (other heights, with NaN, in the areas where you did not measure, will be ignored). Also it accounts for cells which are only partly inside the surveyed area (the volume element will be taken into account proportional to the area (number of gridpoints) covered).

In [ ]:
def calc_vol_under_graph(xxgrid, yygrid, func_3D, height=0.0):
    """
    Calculate the volume (in cubic meters) required to fill the terrain up to a specified height.

    Parameters:
    xxgrid (ndarray): 2D array of x-coordinates from a regular, square meshgrid.
    yygrid (ndarray): 2D array of y-coordinates from a regular, square meshgrid.
    func_3D (ndarray): 2D array of elevation values corresponding to the grid points.
    height (float): Target elevation (NAP) to fill up to. Default is 0.0 meters.

    Returns:
    float: Total volume to fill (positive by convention, indicating material addition).
    """
    # Convert func_3D to a float array to handle NaN and mathematical operations
    func_3D_array = np.array(func_3D, dtype=np.float64)

    # Adjust elevations to compute addition volume
    func_3D_array = height - func_3D_array

    # Calculate cell dimensions (assuming a regular, square grid)
    dx = xxgrid[0, 1] - xxgrid[0, 0]
    dy = yygrid[1, 0] - yygrid[0, 0]
    cell_area = dx * dy

    # Extract the four corners of each cell
    top_left = func_3D_array[:-1, :-1]
    top_right = func_3D_array[:-1, 1:]
    bottom_left = func_3D_array[1:, :-1]
    bottom_right = func_3D_array[1:, 1:]

    # Sum non-NaN values across the four corners for each cell
    sum_non_nan = np.nansum([top_left, top_right, bottom_left, bottom_right], axis=0)

    # Compute total volume: sum over all cells of (sum_non_nan * cell_area / 4)
    volume = np.sum(sum_non_nan * cell_area / 4.0)

    return volume

EWvolume = calc_vol_under_graph(xxnew, yynew, znew, height=0.0)
print('Earthwork volume', EWvolume, 'in m^3')

### Do this yourself

Compute the contour map and 3D DEM for your own dataset using the above code examples, and also compute the volume of Earth work.

In [ ]:
# TODO: Compute the contour map for your dataset

# TODO: Compute the 3D Digital Elevation Model (DEM) for your dataset

# TODO: Calculate the volume of Earthwork for your dataset

**Question 8:** Include the contour map and the 3D DEM in your report.

**Question 9:** Report the volume of Earth work (m^3) to turn the surveyed hill into a flat body at 0 m NAP.

### Actueel Hoogtebestand Nederland (AHN)

Through airborne laser-scanning a 3D Digital Elevation Model (DEM) of the entire of the Netherlands is acquired every couple of years. This DEM can be easily accessed through the AHN-viewer https://www.ahn.nl/ (and choose 'AHN-viewer'). Horizontal coordinates are given in RD, and the vertical coordinate is obviously the NAP-height (all in [m]). The AHN-height is said to be accurate at the 1 decimeter level.

Use the AHN-viewer and zoom-in (to the highest zoom-level, 6 meter) on the little hill on the TU Delft campus that you surveyed with GPS/GNSS during the fieldwork outside. Take a screen-shot and include it in your report. Also compare the AHN DEM with your own DEM, and please comment.

In the AHN-viewer it is also possible to visualize an elevation profile (hoogte-profiel); try the icons in sidebar on the left of the map, to perform several simple terrain analyses. Create an elevation profile West-East, across the little hill and include a screen-shot in your report (you can move the cursor over the elevation profile to inspect the actual height).

**Question 10:** Include the screen-shot of the DEM in the AHN-viewer, as well as the screen-shot of the AHN elevation profile in your report, and comment on the comparison of the AHN with your own DEM.

In [ ]:
# TODO:

[End of the assignment]

[End of Jupyter notebook]